# DS 203 Project
## Potentially Hazardous Asteroid Detection
_________

### III) Predicitve

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Libraries for regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv("final.csv")

XY_cols = ['Absolute Magnitude', 'Eccentricity', 'Inclination', 'Asc Node Longitude', 'Orbital Period',
           'Perihelion Distance', 'Perihelion Arg', 'Aphelion Dist', 'ma', 'rms', 'Mean Motion',
           'Minimum Orbit Intersection', 'Jupiter Tisserand Invariant', 'data_arc']

tdf = df[XY_cols].dropna()

X_cols = ['Absolute Magnitude', 'Eccentricity',  'Inclination', 'Asc Node Longitude', 'Orbital Period',
          'Perihelion Distance', 'Perihelion Arg', 'Aphelion Dist', 'ma', 'rms', 'Mean Motion',
          'Jupiter Tisserand Invariant', 'data_arc']

Y_cols = ['Minimum Orbit Intersection']

X = tdf[X_cols]
Y = tdf[Y_cols]

* Building a simple linear regression model and dropping columns with more less than 20,000 non-null entries.

In [42]:
# using the columns with >20,000 non-null entries
for ratio in [0.1, 0.2, 0.4, 0.6, 0.8, 0.9]:    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=ratio, random_state=32)
#     sc = StandardScaler()
#     X_train = sc.fit_transform(X_train)
#     X_test = sc.transform(X_test)
    lin_model = LinearRegression()
    lin_model.fit(X_train, Y_train)

    prediction = lin_model.predict(X_test)
    
    MSE = mean_squared_error(Y_test, prediction)
    r2 = r2_score(Y_test, prediction)

    print('Model Performance (Train test ratio = {}):\n-------------------------------------------'.format(ratio))
    print('Mean Squared Error:', MSE)
    print('R^2 value:', r2)
    print('-----------------------------------------------------', '\n')

Model Performance (Train test ratio = 0.1):
-------------------------------------------
Mean Squared Error: 0.004820563285114296
R^2 value: 0.5385938735615758
----------------------------------------------------- 

Model Performance (Train test ratio = 0.2):
-------------------------------------------
Mean Squared Error: 0.0048343294453212955
R^2 value: 0.538431182864496
----------------------------------------------------- 

Model Performance (Train test ratio = 0.4):
-------------------------------------------
Mean Squared Error: 0.0048122816607502
R^2 value: 0.539397204911691
----------------------------------------------------- 

Model Performance (Train test ratio = 0.6):
-------------------------------------------
Mean Squared Error: 0.004796638570589563
R^2 value: 0.5372109842462014
----------------------------------------------------- 

Model Performance (Train test ratio = 0.8):
-------------------------------------------
Mean Squared Error: 0.004886410682136869
R^2 value: 0.5

#### Conclusions:
* The low $R^2$ scores for different combinations of train-test ratios suggest that simple linear regression is not a good model for predicting the Minimum Orbit Intersection of the asteroid.
* We now include columns with atlest 4,000 non-null entries.

In [45]:
# using columns with > 4,000 non-null entries
cont_cols = ['Absolute Magnitude', 'Relative Velocity km per sec', 'Miss Dist.(kilometers)', 'Minimum Orbit Intersection',
             'Jupiter Tisserand Invariant', 'Eccentricity', 'Inclination', 'Asc Node Longitude',
             'Orbital Period', 'Perihelion Distance', 'Perihelion Arg', 'Aphelion Dist', 'Mean Motion',
             'ma', 'data_arc', 'rms', "diameter"] 

tdf = df[cont_cols].dropna()

X_all_cols = ['Eccentricity', 'Inclination', 'Asc Node Longitude', 'Orbital Period', 'Perihelion Distance', 'Perihelion Arg',
     'Aphelion Dist', 'ma', 'rms', 'Mean Motion']

Y_all_cols = ['Minimum Orbit Intersection']

X_all = tdf[X_all_cols]
Y_all = tdf[Y_all_cols]
for ratio in [0.1, 0.2, 0.4, 0.6, 0.8, 0.9]:    
    X_train, X_test, Y_train, Y_test = train_test_split(X_all, Y_all, train_size=ratio, random_state=42)
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    lin_model = LinearRegression()
    lin_model.fit(X_train, Y_train)

    prediction = lin_model.predict(X_test)
    
    MSE = mean_squared_error(Y_test, prediction)
    r2 = r2_score(Y_test, prediction)

    print('Model Performance (Train test ratio = {}):\n-------------------------------------------'.format(ratio))
    print('Mean Squared Error:', MSE)
    print('R^2 value:', r2)
    print('-----------------------------------------------------', '\n')

Model Performance (Train test ratio = 0.1):
-------------------------------------------
Mean Squared Error: 0.0053397732706093205
R^2 value: 0.3677541278547125
----------------------------------------------------- 

Model Performance (Train test ratio = 0.2):
-------------------------------------------
Mean Squared Error: 0.005240843255687071
R^2 value: 0.37439957624452647
----------------------------------------------------- 

Model Performance (Train test ratio = 0.4):
-------------------------------------------
Mean Squared Error: 0.005201469448426365
R^2 value: 0.3627151263340561
----------------------------------------------------- 

Model Performance (Train test ratio = 0.6):
-------------------------------------------
Mean Squared Error: 0.005077666028748927
R^2 value: 0.3741095568570546
----------------------------------------------------- 

Model Performance (Train test ratio = 0.8):
-------------------------------------------
Mean Squared Error: 0.005050100513694052
R^2 value

#### Conclusions:
* The poor $R^2$ scores in this case too indicate that a simple linear model is not ideal to capture the replationship between the Minimum Orbit Intersection of the asteroid and the other parameters.
* We now proceed to try adding a $L_1$(Ridge) and an $L_2$(Lasso) loss during our optimisation. 

In [46]:
for ratio in [0.2, 0.4, 0.6, 0.8, 0.9]:
    for lamda in [0.001, 0.01, 0.1]:        
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=ratio, random_state=32)

        ridge_model = Ridge(alpha=lamda, tol=0.01)
        ridge_model.fit(X_train, Y_train)
        
        lasso_model = Lasso(alpha=lamda, tol=0.01)
        lasso_model.fit(X_train, Y_train)

        prediction_ridge = ridge_model.predict(X_test)
        prediction_lasso = lasso_model.predict(X_test)

        MSE_ridge = mean_squared_error(Y_test, prediction_ridge)
        r2_ridge = r2_score(Y_test, prediction_ridge)
        
        MSE_lasso = mean_squared_error(Y_test, prediction_lasso)
        r2_lasso = r2_score(Y_test, prediction_lasso)

        print('Model Performance (Train test ratio = {}, λ = {}):\n-------------------------------------------'.format(ratio, lamda))
        print('Mean Squared Error (Lasso):', MSE_lasso)
        print('R^2 value          (Lasso):', r2_lasso)
        print('Mean Squared Error (Ridge):', MSE_ridge)
        print('R^2 value          (Ridge):', r2_ridge)
        
        print('-----------------------------------------------------', '\n')

Model Performance (Train test ratio = 0.2, λ = 0.001):
-------------------------------------------
Mean Squared Error (Lasso): 0.005131025336717952
R^2 value          (Lasso): 0.5101034544401422
Mean Squared Error (Ridge): 0.004834301838912826
R^2 value          (Ridge): 0.5384338186503783
----------------------------------------------------- 

Model Performance (Train test ratio = 0.2, λ = 0.01):
-------------------------------------------
Mean Squared Error (Lasso): 0.006578003188634326
R^2 value          (Lasso): 0.37194988772848725
Mean Squared Error (Ridge): 0.004834057728578381
R^2 value          (Ridge): 0.5384571256507793
----------------------------------------------------- 

Model Performance (Train test ratio = 0.2, λ = 0.1):
-------------------------------------------
Mean Squared Error (Lasso): 0.008392294004583935
R^2 value          (Lasso): 0.1987262637845023
Mean Squared Error (Ridge): 0.00483200436055724
R^2 value          (Ridge): 0.5386531757254343
------------------

#### Conclusions:
* The poor $R^2$ scores for different values of λ and train-test ratios for these Lasso and Ridge regression models suggest that the Minimum orbit intersection of the asteroid has a non-linear dependance on the other parameters.
* We now try a Random Forest model to predict the dependance of the Minimum orbit intersection of the asteroid with the other parameters.

In [6]:
XY_cols = ['Absolute Magnitude', 'Eccentricity', 'Inclination', 'Asc Node Longitude', 'Orbital Period',
           'Perihelion Distance', 'Perihelion Arg', 'Aphelion Dist', 'ma', 'rms', 'Mean Motion',
           'Minimum Orbit Intersection', 'Jupiter Tisserand Invariant', 'data_arc']

tdf = df[XY_cols].dropna()

X_cols = ['Absolute Magnitude', 'Eccentricity',  'Inclination', 'Asc Node Longitude', 'Orbital Period',
          'Perihelion Distance', 'Perihelion Arg', 'Aphelion Dist', 'ma', 'rms', 'Mean Motion',
          'Jupiter Tisserand Invariant', 'data_arc']

Y_cols = ['Minimum Orbit Intersection']

X = np.array(tdf[X_cols])
Y = np.array(tdf[Y_cols]).ravel()

In [7]:
for ratio in [0.5, 0.7, 0.9]:    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=ratio)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    for n_trees in [40, 60, 80, 100, 150]:
        forest = RandomForestRegressor(n_estimators=n_trees, random_state=0)
        forest.fit(X_train, Y_train)
        prediction = forest.predict(X_test)

        MSE = mean_squared_error(Y_test, prediction)
        r2 = r2_score(Y_test, prediction)

        print('Model Performance (n_estimators = {}, train ratio =  {}):\n-------------------------------------------'.format(n_trees, ratio))
        print('Mean Squared Error:', MSE)
        print('R^2 value:', r2)
        print('-----------------------------------------------------', '\n')

Model Performance (n_estimators = 40, train ratio =  0.5):
-------------------------------------------
Mean Squared Error: 0.0009577826672636222
R^2 value: 0.9064441542869404
----------------------------------------------------- 

Model Performance (n_estimators = 60, train ratio =  0.5):
-------------------------------------------
Mean Squared Error: 0.0009323162682944043
R^2 value: 0.9089317024273116
----------------------------------------------------- 

Model Performance (n_estimators = 80, train ratio =  0.5):
-------------------------------------------
Mean Squared Error: 0.000916774894004228
R^2 value: 0.9104497779416812
----------------------------------------------------- 

Model Performance (n_estimators = 100, train ratio =  0.5):
-------------------------------------------
Mean Squared Error: 0.0009149953267852251
R^2 value: 0.9106236053890423
----------------------------------------------------- 

Model Performance (n_estimators = 150, train ratio =  0.5):
----------------

#### Conclusions:
* The high $R^2$ scores and low MSE indicate that the Random Forest model is reliable and can be used to accurately predict the Minimum orbit intersection of the asteroid from the gven parameters.